# Constraint and Guided Generation Technique
The problem in LLM is while generating output they sometimes produce output open-ended or lack of desired characteristic. SO we use `Constarint and Guided generation Technique` which allow to control over model's output making more suitable for specific tasks or adhering to certain rules and formats.

In [ ]:
! pip install langchain langchain-google-genai

In [2]:
import os
import re
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import RegexParser

# Initialize the GOOGLE_API_KEY
os.environ['GOOGLE_API_KEY']='GOOGLE_API_KEY'

# Initialize the LLM
llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# Function to display model Output
def display_output(output):
  """Display LLM the Output in formatted manner"""
  print('Model Output:')
  print("*"*40)
  print(output)
  print("*"*40)
  print()

# Constarints for Model Output
You guide the model by placing explicit limits or rule on its output.
* Enforce structure
* Limit response length
* Avoid certain topoics or words
* Match a format or tone

In [3]:
constrained_prompt=PromptTemplate(
    input_variables=['product','target_audience','tone','word_limit'],
    template  ="""
        Create product description for {product} targeted at {target_audience}.
        Use a {tone} tone and keep it under {word_limit} word.
        The description should include:
        1. A catchy headline
        2. Three key features
        3. A call to action

        Product Description:

    """
)

# Generate the constraint output
input_variables={
    'product':'Smart water bottle',
    'target_audience':'health conscious milennieals',
    'tone':'casual and friendly',
    'word_limit':'75'
}

# Form a chain
chain = constrained_prompt | llm

# Generate response
output=chain.invoke(input_variables).content
display_output(output)

Model Output:
****************************************
**Stay Hydrated, Slay Your Goals!**

This smart water bottle tracks your daily intake, reminds you to drink up, and even glows to celebrate your wins!  It's sleek, stylish, and helps you crush your hydration goals.  Plus, it syncs with your favorite fitness apps.  Get yours today and level up your wellness game!
****************************************



# Rule Based Generation
You provide the gentle direction or contextual framing to guide the tone style or content.

In [4]:
job_posting_prompt=PromptTemplate(
    input_variables=['job_title','company','location','experience'],
    template="""
        Create a job posting {job_title} position at {company} in {location}.
        The candidate shoudl have {experience} years of experience.
        Follow these rules:
          1. Start with a brief company description (2 sentences)
          2. List 5 key responsibilities, each starting with an action verb
          3. List 5 required qualifications, each in a single sentence
          4. End with a standardized equal opportunity statement

          Format the output as follows:
          COMPANY: [Company Description]

          RESPONSIBILITIES:
          - [Responsibility 1]
          - [Responsibility 2]
          - [Responsibility 3]
          - [Responsibility 4]
          - [Responsibility 5]

          QUALIFICATIONS:
          - [Qualification 1]
          - [Qualification 2]
          - [Qualification 3]
          - [Qualification 4]
          - [Qualification 5]

    EEO: [Equal Opportunity Statement]

    """
)

# Generate rule based output
input_variables={
    'job_title':'Senior Data Scientiest',
    'company':'IBM',
    'location':'Banglore, India',
    'experience':'3+'
}

# Form a chain
chain = job_posting_prompt | llm

# Generate response
output= chain.invoke(input_variables).content
display_output(output)

Model Output:
****************************************
COMPANY: IBM is a global technology leader, providing innovative solutions to businesses and individuals worldwide.  We are committed to creating a diverse and inclusive workplace where everyone can thrive.

RESPONSIBILITIES:
- Develop and implement advanced machine learning models to solve complex business problems.
- Collaborate with cross-functional teams to define data science projects, gather requirements, and deliver impactful insights.
- Design, build, and maintain robust and scalable data pipelines to support data analysis and model training.
- Communicate complex technical concepts clearly and effectively to both technical and non-technical audiences.
- Stay abreast of the latest advancements in data science and apply them to improve existing solutions and develop new capabilities.


QUALIFICATIONS:
- Possess a Master's degree or PhD in a quantitative field such as Computer Science, Statistics, Mathematics, or a related di

# Using Regex parser for structured output

In [7]:
# Define a regex parser for structured output
regex_parser = RegexParser(
    regex=r"COMPANY:\s*([\s\S]*?)\n\s*RESPONSIBILITIES:\s*([\s\S]*?)\n\s*QUALIFICATIONS:\s*([\s\S]*?)\n\s*EEO:\s*([\s\S]*)",
    output_keys=["company_description", "responsibilities", "qualifications", "eeo_statement"]
)
# This regex pattern captures the company description, responsibilities, qualifications, and EEO statement from the output text.

# Create a new prompt template that includes the parser instructions
parsed_job_posting_prompt = PromptTemplate(
    input_variables=["job_title", "company", "location", "experience"],
    template="""Create a job posting for a {job_title} position at {company} in {location}.
    The candidate should have {experience} years of experience.
    Follow these rules:
    1. Start with a brief company description (2 sentences)
    2. List 5 key responsibilities, each starting with an action verb
    3. List 5 required qualifications, each in a single sentence
    4. End with a standardized equal opportunity statement

    Format the output EXACTLY as follows:
    COMPANY: [Company Description]

    RESPONSIBILITIES:
    - [Responsibility 1]
    - [Responsibility 2]
    - [Responsibility 3]
    - [Responsibility 4]
    - [Responsibility 5]

    QUALIFICATIONS:
    - [Qualification 1]
    - [Qualification 2]
    - [Qualification 3]
    - [Qualification 4]
    - [Qualification 5]

    EEO: [Equal Opportunity Statement]
    """
)

def clean_output(output):
    for key, value in output.items():
        if isinstance(value, str):
            # Remove leading/trailing whitespace and normalize newlines
            output[key] = re.sub(r'\n\s*', '\n', value.strip())
    return output

# Generate the parsed output
chain = parsed_job_posting_prompt | llm
raw_output = chain.invoke(input_variables).content

# Parse and clean the output
parsed_output = regex_parser.parse(raw_output)
cleaned_output = clean_output(parsed_output)

# Display the parsed output
print("Parsed Output:")
for key, value in cleaned_output.items():
    print(f"{key.upper()}:")
    print(value)
    print()

Parsed Output:
COMPANY_DESCRIPTION:
IBM is a global technology leader, providing innovative solutions to clients worldwide.  We are committed to creating a diverse and inclusive workplace where everyone can thrive.

RESPONSIBILITIES:
- Develop and implement advanced statistical models and machine learning algorithms to solve complex business problems.
- Collaborate with cross-functional teams, including engineers, product managers, and business stakeholders, to define data science projects and deliver actionable insights.
- Design and build robust and scalable data pipelines for efficient data ingestion, processing, and analysis.
- Communicate complex technical findings to both technical and non-technical audiences through clear and concise presentations and reports.
- Stay abreast of the latest advancements in data science and machine learning, and apply them to improve the effectiveness of our solutions.

QUALIFICATIONS:
- You possess a Master's degree or PhD in a quantitative field 

In [8]:
review_prompt = PromptTemplate(
    input_variables=["product", "rating", "pros", "cons", "word_limit"],
    template="""Write a product review for {product} with the following constraints:
    1. The review should have a {rating}-star rating (out of 5)
    2. Include exactly {pros} pros and {cons} cons
    3. Use between 2 and 3 sentences for each pro and con
    4. The entire review should be under {word_limit} words
    5. End with a one-sentence recommendation

    Format the review as follows:
    Rating: [X] out of 5 stars

    Pros:
    1. [Pro 1]
    2. [Pro 2]
    ...

    Cons:
    1. [Con 1]
    2. [Con 2]
    ...

    Recommendation: [One-sentence recommendation]
    """
)

# Generate the constrained review
input_variables = {
    "product": "Smartphone X",
    "rating": "4",
    "pros": "3",
    "cons": "2",
    "word_limit": "200"
}

chain = review_prompt | llm
output = chain.invoke(input_variables).content
display_output(output)

Model Output:
****************************************
Rating: 4 out of 5 stars

Pros:
1. The camera is excellent.  Photos are crisp and clear, even in low light conditions, and the video recording is smooth and stable.  I've been very impressed with its versatility.
2. The battery life is outstanding. I easily get a full day of use, even with heavy usage, and it charges relatively quickly. This is a major plus for me.
3. The display is vibrant and sharp.  Colors are rich and viewing angles are excellent, making it a joy to watch videos or browse photos. It's one of the best screens I've used on a phone.

Cons:
1. The phone is quite slippery and I've nearly dropped it a few times. A case is a must-have accessory.
2. The price point is a bit high compared to some competitors offering similar specs.  While it's worth it for the features, it's not the most budget-friendly option.

Recommendation: Smartphone X is a great phone with some minor drawbacks, making it a solid choice for most us